In [1]:
import pandas as pd 
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import backtrader as bt
import datetime
import os
from flask import Flask, render_template, request, redirect, url_for, flash


In [73]:
ticker = 'NVDA'
# getting the last 5 years of data until current date
data = yf.download(ticker, start = datetime.datetime.now() - datetime.timedelta(1500), end = datetime.datetime.now())
data.columns = data.columns.droplevel([1]) # clear the multi-index columns
data_feed = bt.feeds.PandasData(dataname = data)

[*********************100%***********************]  1 of 1 completed


In [74]:
carebro = bt.Cerebro()# the class that will handle the backtesting 
carebro.adddata(data_feed)  # carebro takes in a feed from a backtrader pandas DF 
carebro.broker.setcash(10000) # might want to expose that to the user later

In [75]:
class SmaCrossStrategy(bt.Strategy):
    params = (('fast', 1),('slow', 200),('mega_slow', 500), ) # parameters for the strategy
    def __init__(self):
        self.sma_fast = bt.indicators.SimpleMovingAverage(self.datas[0], period = self.params.fast)
        self.sma_slow = bt.indicators.SimpleMovingAverage(self.datas[0], period = self.params.slow)
        self.sma_super_slow = bt.indicators.SimpleMovingAverage(self.datas[0], period = self.params.mega_slow)
        self.crossover_fs = bt.indicators.CrossOver(self.sma_fast, self.sma_slow) # crossover signal
        self.crossover_ss = bt.indicators.CrossOver(self.sma_slow, self.sma_super_slow) # crossover signal

# add more indicators classes here RSI, MACD, etc.

    def next(self):
        # if we are not in a position and a golder cross occurs, buy
        if not self.position: # self.postion = 0 when we are not in a position
            if self.crossover_fs > 0: # self.crossover_fs = 1 when the fast sma crosses the slow sma from below
                self.buy()  # buy the stock
        else : 
            if self.crossover_fs < 0: # self.crossover_fs = -1 when the fast sma crosses the slow sma from above  
                self.sell()
        # add more conditions here for more indicators
            

In [76]:
class buy_and_hold(bt.Strategy):
    def next(self):
        self.buy()
carebro2 = bt.Cerebro()# the class that will handle the backtesting 
carebro2.adddata(data_feed)  # carebro takes in a feed from a backtrader pandas DF 
carebro2.broker.setcash(10000) # might want to expose that to the user later
carebro2.addstrategy(buy_and_hold)
carebro2.addsizer(bt.sizers.PercentSizer, percents = 100) # 10% of the cash will be used for each trade
results2 = carebro2.run()

In [77]:
carebro.addstrategy(SmaCrossStrategy)
carebro.addsizer(bt.sizers.PercentSizer, percents = 10) # 10% of the cash will be used for each trade
carebro.addanalyzer(bt.analyzers.SharpeRatio, _name = 'sharpe')
results = carebro.run()


In [78]:
final_value = carebro.broker.getvalue()
final_value2 = carebro2.broker.getvalue()
print(f'Final value from strat1 : {final_value}')
print(f'Final value from strat2 : {final_value2}')
sharp = results[0].analyzers.sharpe.get_analysis()

Final value from strat1 : 9834.291248292799
Final value from strat2 : 73857.92715901606


NameError: name 'strategy' is not defined

In [ ]:
datetime.datetime.now()
# Create a Flask applicatio

datetime.datetime(2025, 4, 1, 15, 21, 32, 772346)

In [3]:
datetime.timedelta(10)

datetime.timedelta(days=10)

In [24]:
class Student: 
    def __init__(self):
        self.name = 'Emil'

In [25]:
student = Student()

In [35]:
class Store:
    def __init__(self, name, items: list):
        self.name = name 
        self.items = items
        # You'll need 'name' as an argument to this method.
        # Then, initialise 'self.name' to be the argument, and 'self.items' to be an empty list.
    
    def add_item(self, name, price):
        self.items.append({'name' : name, 'price' : price})
        # Create a dictionary with keys name and price, and append that to self.items.

    def stock_price(self):
        total = 0
        for i in self.items:
            total = total + i['price']
        return total
        # Add together all item prices in self.items and return the total.

store1 = Store('lexi', [])
store1.add_item('fanta', 2)
store1.add_item('cola', 6)
store1.stock_price()


8

In [74]:
import functools
user = {
    'username' : 'Emil',
    'access_lvl' : 'admin' 
}

def make_secure(access_level):
    def decorator(func):
        @functools.wraps(func)
        def secure_function(*args, **kwargs):
            if user['access_lvl'] == access_level:
                return func(*args, **kwargs)
            else:
                print('insufissient priv')
        return secure_function
    return decorator


@make_secure('admin')
def get_admin_pw():
    return 1234

print(get_admin_pw.__defaults__)


None


get_admin_pw
